<a href="https://colab.research.google.com/github/MVolobueva/MTase-classification/blob/main/Classification_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MTase classification
In this notebook we will focus on the pipeline for MTase classification.

*Our classification algorithm   uses cat-profiles (HMM profiles that locate catalytic motifs from DB Pfam and SUPFAM). Cat-profiles allows detecting regions of cat- and sam- subdomains and motifs.*

Pipeline involves 3 steps:


1.   Running hmmer package
2.   Cutting subdomain and motif region from profile alignment and MTase sequence
3.  Class detection on motif and subdomain regions




In [ ]:
!rm -r MTase-classification/

##0. Clone github repository

In [ ]:
!git clone https://github.com/MVolobueva/MTase-classification.git

Cloning into 'MTase-classification'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 21 (delta 1), reused 21 (delta 1), pack-reused 0
Receiving objects: 100% (21/21), 460.69 KiB | 1.51 MiB/s, done.
Resolving deltas: 100% (1/1), done.


## 1.Running hmmer package



### 1.1 Hmmer packalge instalation

In [ ]:
!sudo apt-get -y install hmmer

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdivsufsort3
Suggested packages:
  hmmer-doc
The following NEW packages will be installed:
  hmmer libdivsufsort3
0 upgraded, 2 newly installed, 0 to remove and 9 not upgraded.
Need to get 1,198 kB of archives.
After this operation, 7,621 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libdivsufsort3 amd64 2.0.1-5 [42.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 hmmer amd64 3.3.2+dfsg-1 [1,155 kB]
Fetched 1,198 kB in 2s (589 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requi

### 1.2 HMM profiles search

Make foldor for results

In [ ]:
!mkdir ./results/

**file.stk**  - file with alignment of MTase sequences according to HMM profile

**file.domtbl** - file with HMM profile hits


#### 1.2.1 SUPFAM profiles search

In [ ]:
!hmmsearch --cpu 3 -E 0.01 --domE 0.01 --incE 0.01 --incdomE 0.01 \
        -o /dev/null --noali -A ./results/SUPFAM.stk --domtblout ./results/SUPFAM.domtbl \
        ./MTase-classification/HMM_profiles/cat_profiles_SUPFAM.hmm ./MTase-classification/Sample_MTases/rebenz.txt

###2. Cutting subdomain and motif region from profile alignment and MTase sequence

Install required libraries


In [ ]:
!git clone https://github.com/isrusin/etsv
!python3 -m pip install -e etsv

Cloning into 'etsv'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 47 (delta 10), reused 47 (delta 10), pack-reused 0
Receiving objects: 100% (47/47), 9.92 KiB | 4.96 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Obtaining file:///content/etsv
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for etsv (pyproject.toml) ... done
  Created wheel for etsv: filename=etsv-0.0.2-0.editable-py3-none-any.whl size=1282 sha256=26a6d784e5389b23f2012e0e32e9989a449eca168b175a88ea0be26d09fd5ca7
  Stored in directory: /tmp/pip-ephem-wheel-cache-wllwii0u/wheels/54/37/04/f3f5689d9aa16adc2b15b6be1fe70879c9be31cb99bdf04281
Successfully built etsv


Cut sam- and cat- subdomain and motif regions from alignments

In [ ]:
!./MTase-classification/Scripts/get_aln_regions.py \
  ./MTase-classification/profile-markup/SMD_regions.tsv \
  ./results/SUPFAM.stk > ./results/region_alignments_SUPFAM.tsv

**region_alignments.tsv** - file with cut sections of regions from the MTase
sequence alignment along a cat-profile

##3. Detect class

In [ ]:
!mkdir ./results/SUPFAM

In [ ]:
!python ./MTase-classification/Scripts/classification_SUPFAM.py

Path to dataframe:./results/region_alignments_SUPFAM.tsv
Path to cat-motif from class A:/content/MTase-classification/Scripts/A_class.csv
Path to cat-motif from class B:/content/MTase-classification/Scripts/B_class.csv
Path to cat-motif from class C:/content/MTase-classification/Scripts/C_class.csv
Path to cat-motif from class D:/content/MTase-classification/Scripts/D_class.csv
where to save:./results/SUPFAM
3
3
3
3
3
3


In [ ]:
!python ./MTase-classification/Scripts/classification_SUPFAM.py


Path to dataframe:./results/region_alignments_SUPFAM.tsv
Path to cat-motif from class A:/content/MTase-classification/Scripts/A_class.csv
Path to cat-motif from class B:/content/MTase-classification/Scripts/B_class.csv
Path to cat-motif from class C:/content/MTase-classification/Scripts/C_class.csv
Path to cat-motif from class D:/content/MTase-classification/Scripts/D_class.csv
where to save:./results/SUPFAM
3
3
3
3
3
3
5
5
5
5


In [ ]:
import pandas as pd
pd.read_csv('/content/results/SUPFAM/classes.csv')

,Unnamed: 0,REBASE_name,A_topology,B_topology,C_topology,D_topology,K_topology,M_topology,L_topology
0,0,BaeI,435-438,NaN,NaN,NaN,NaN,NaN,NaN
